# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
from datetime import datetime
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
from typing import List, Dict, Annotated
from pydantic import BaseModel ,Field
from lib.vector_db import VectorStoreManager
from tavily import TavilyClient

In [3]:
load_dotenv()

assert os.getenv("OPENAI_API_KEY") is not None
assert os.getenv("OPENAI_BASE_URL") is not None
assert os.getenv("TAVILY_API_KEY") is not None

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

## Init VectorStoreManager and init Store

- It using inside the chromadb and chroma_client lib

In [4]:
manager = VectorStoreManager(name="chromadb", 
                             openai_api_key=OPENAI_API_KEY, 
                             openai_base_url=OPENAI_BASE_URL)


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
class GameDocument(BaseModel):
    """
    Game document containing information about a video game.
    
    Each element contains:
        - Platform: like Game Boy, Playstation 5, Xbox 360...
        - Name: Name of the Game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
    """
    Platform: Annotated[str, Field(description="Platform: like Game Boy, Playstation 5, Xbox 360...)")]
    Name: Annotated[str, Field(description="Name of the Game")]
    YearOfRelease: Annotated[int, Field(description="Year when that game was released for that platform")]
    Description: Annotated[str, Field(description="Additional details about the game")]

In [6]:
@tool
def retrieve_game (query:str, n_results: int = 3) -> List[GameDocument]:
    """
    Semantic search: Finds most results in the vector DB.
    
    Args:
        query: A question about game industry.
    
    Returns:
        List[GameDocument]: List of game documents matching the query.
    """
    
    chroma_client = manager.get_or_create_store("udaplay")
    
    results = chroma_client.query(query_texts=query, n_results=n_results)

    return [GameDocument(**meta) for meta in results['metadatas'][0]]

In [7]:
@tool
def retrieve_all_games()->List[GameDocument]:

    chroma_client = manager.get_or_create_store("udaplay")

    results = chroma_client.get()

    return [GameDocument(**meta) for meta in results['metadatas'][0]]

#### Evaluate Retrieval Tool

In [8]:
class GameEvaluation(BaseModel):
    useful: Annotated[bool, Field(description="whether the documents are useful to answer the question")]
    description: Annotated[str, Field(description="description about the evaluation result")]

In [9]:
@tool
def evaluate_retrieval(question:str, retrieved_docs:List[GameDocument])->GameEvaluation:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question.
    args: 
        - question: original question from user
        - retrieved_docs: Full context of documents available to answer the question
    The result includes:
        - useful: whether the documents are useful to answer the question
        - description: description about the evaluation result
    """

    llm = LLM( api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

    system_prompt = """
        
        You are Professor Eval, a critical evaluation expert specializing 
        in information sufficiency analysis.

        Your task: Assess whether the provided documents contain sufficient information to fully and 
        accurately answer the user's question.

        Evaluation Guidelines:
            - Be strict and objective - base your judgment solely on the provided documents
            - Check if all necessary facts, details, and context are present
            - Identify any gaps, ambiguities, or missing information
            - Do not assume information beyond what is explicitly provided

        Set `useful`:
            - True: Only if the documents contain complete and clear information to answer the question fully
            - False: If any critical information is missing, unclear, insufficient, unclear or requires assumptions

        Provide `description`:
            - A concise 2-3 sentence explanation of your evaluation
            - State what is present or what is missing
            - Be direct and specific

        When in doubt, mark as False. Quality over leniency.
    """
    
    user_prompt = f"""

        USER QUESTION:
        {question}

        RETRIEVED DOCUMENTS:
        {retrieved_docs}

        TASK: 
        Evaluate whether the retrieved documents contain sufficient information to fully answer the user question.
    """

    messages=[
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    return llm.invoke( input=messages, response_format=GameEvaluation).content


#### Game Web Search Tool

In [10]:
class SearchResult(BaseModel):
    answer: Annotated[str, Field(description="web search answer to the query")]
    results: Annotated[List[str], Field(description="web search results as list")]
    timestamp: str = Field(default_factory=lambda: datetime.now().isoformat(), description="timestamp")
    query: Annotated[str, Field(description="initial query")]

In [11]:
@tool
def game_web_search(question: str) -> SearchResult:
    """
     Semantic search: Finds most results in the vector DB
    args:
        - question: a question about game industry. 
    """
    search_client = TavilyClient(api_key=TAVILY_API_KEY)

    results = search_client.search(query=question, 
                                   search_depth="advanced",
                                   include_answer=True,
                                   include_favicon=False,
                                   include_images=False,
                                   include_raw_content=False
                                   )
    
    return SearchResult(answer=results.get("answer", ""),
                results=results.get("results", []),
                query=question)

### Agent

In [12]:
instructions = """

You are a knowledgeable video game expert assistant specializing in providing accurate 
and comprehensive information about computer games.

Your capabilities:
- Search and retrieve detailed game information from a knowledge base
- Evaluate whether retrieved information is sufficient to answer questions
- Perform web searches for additional or up-to-date game information when needed

Your approach:
1. Always start by retrieving information from the knowledge base
2. Critically evaluate if the retrieved information fully answers the user's question
3. If information is insufficient or outdated, perform a web search to supplement your answer
4. Provide clear, accurate, and well-structured responses
5. Cite your sources when providing specific facts or data

Guidelines:
- Be precise and factual - avoid speculation
- If information is unavailable, clearly state this
- For questions about recent games or updates, prioritize web search results
- Combine multiple sources when needed for comprehensive answers
- Keep responses concise but informative

Your goal is to provide the most accurate and helpful game information possible.

"""

gameAgent = Agent(
    model_name="gpt-4o",
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions=instructions
)

In [13]:
def print_conversation(messages):
    for i, msg in enumerate(messages, 1):
        role = msg.role.upper()
        
        if isinstance(msg, AIMessage) and msg.tool_calls:
            tool_names = [tc.function.name for tc in msg.tool_calls]
            print(f"[{i}] 🤖 {role}: Tool calls → {', '.join(tool_names)}")
        elif isinstance(msg, ToolMessage):
            content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
            print(f"[{i}] 🔧 TOOL ({msg.name}): {content_preview}")
        elif isinstance(msg, SystemMessage):
            print(f"[{i}] ⚙️  {role}: [System instructions]")
        else:
            print(f"[{i}] {'👤' if role == 'USER' else '🤖'} {role}: {msg.content}")
        print()

### First Question

In [14]:
query = "When Pokémon Gold and Silver was released?"

response = gameAgent.invoke(query=query)

print(print_conversation(response.get_final_state()["messages"]))

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[1] ⚙️  SYSTEM: [System instructions]

[2] 👤 USER: When Pokémon Gold and Silver was released?

[3] 🤖 ASSISTANT: Tool calls → retrieve_game

[4] 🔧 TOOL (retrieve_game): "[GameDocument(Platform='Game Boy Color', Name='Pok\u00e9mon Gold and Silver', YearOfRelease=1999, D...

[5] 🤖 ASSISTANT: Tool calls → evaluate_retrieval

[6] 🔧 TOOL (evaluate_retrieval): "{\"useful\":true,\"description\":\"The documents clearly state that Pok\u00e9mon Gold and Silver we...

[7] 🤖 ASSISTANT: Pokémon Gold and Silver were released in 1999 for the Game Boy Color. These games are part of the second generation of Pokémon games, introducing new regions, Pokémon, and ga

### Second Question

In [16]:
query = "Which one was the first 3D platformer Mario game?"

response = gameAgent.invoke(query=query)

print(print_conversation(response.get_final_state()["messages"]))

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[1] ⚙️  SYSTEM: [System instructions]

[2] 👤 USER: When Pokémon Gold and Silver was released?

[3] 🤖 ASSISTANT: Tool calls → retrieve_game

[4] 🔧 TOOL (retrieve_game): "[GameDocument(Platform='Game Boy Color', Name='Pok\u00e9mon Gold and Silver', YearOfRelease=1999, D...

[5] 🤖 ASSISTANT: Tool calls → evaluate_retrieval

[6] 🔧 TOOL (evaluate_retrieval): "{\"useful\":true,\"description\":\"The documents clearly state that Pok\u00e9mon Gold and Silver we...

[7] 🤖 ASSISTANT: Pokémon Gold and Silver were released in 1999 for the Game Boy Color. These games are part of the second generation of Pokémon games, introducing new regions, Pokémon, and ga

### Third Question

In [17]:
query = "Was Mortal Kombat X realeased for Playstation 5?"

response = gameAgent.invoke(query=query)

print(print_conversation(response.get_final_state()["messages"]))

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor


ValidationError: 5 validation errors for SearchResult
results.0
  Input should be a valid string [type=string_type, input_value={'url': 'https://www.psu....39, 'raw_content': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
results.1
  Input should be a valid string [type=string_type, input_value={'url': 'https://store.pl...05, 'raw_content': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
results.2
  Input should be a valid string [type=string_type, input_value={'url': 'https://gamefaqs...48, 'raw_content': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
results.3
  Input should be a valid string [type=string_type, input_value={'url': 'https://www.play...22, 'raw_content': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
results.4
  Input should be a valid string

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes